In [ ]:
!pip install ultralytics 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.8/898.8 kB 31.5 MB/s eta 0:00:00


In [2]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np 
import pandas as pd 

import cv2
import pydicom
from PIL import Image
from IPython.display import Image as IPyImage, display

import os
import re
import glob
import random
from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="whitegrid")

import torch
from torch import nn
import torch.nn.functional as F
from torch.optim import AdamW
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingLR

from sklearn.model_selection import train_test_split

from torchvision import transforms
import timm

import yaml

import albumentations as A

from sklearn.model_selection import KFold

from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
train_df = pd.read_csv('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train.csv')
label_coords_df = pd.read_csv('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_label_coordinates.csv')
series_desc_df = pd.read_csv('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_series_descriptions.csv')

In [ ]:
# Getting a list of all the study IDs and paths to their images
images_dir_path = r'/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images'
study_id_list = os.listdir(images_dir_path)
study_id_paths = [(x, f"{images_dir_path}/{x}") for x in study_id_list]

# Initialize the metadata dictionary
meta_df = {}

# Process each study and its series
for study_id, study_folder_path in study_id_paths:
    series_ids = []
    series_descriptions = []
    
    # Get all the series IDs (folders) within the study folder
    try:
        series_folders = os.listdir(study_folder_path)
    except FileNotFoundError as e:
        print(f"Error: Folder not found for study {study_id}. Skipping this study.")
        continue  # Skip this study if the folder doesn't exist

    # Process each series in the study folder
    for series_id in series_folders:
        try:
            # Fetch the series description from the dataframe
            series_description = series_desc_df[series_desc_df['series_id'] == int(series_id)]['series_description'].iloc[0]
        except (IndexError, ValueError):
            # Handle cases where series_id is not found in the dataframe or can't be converted to int
            series_description = 'Unknown'

        # Append series ID and description to the lists
        series_ids.append(series_id)
        series_descriptions.append(series_description)
    
    # Add metadata for the current study_id
    meta_df[int(study_id)] = {
        'folder_path': study_folder_path,
        'series_ids': series_ids,
        'series_descriptions': series_descriptions
    }

    

In [5]:
def add_desc(df, meta_df):
    df['series_desc'] = None

    # Iterate over rows in the dataframe
    for idx, coor_row in df.iterrows():
        try:
            # Find the meta_df for the study_id
            meta_info = meta_df[int(coor_row['study_id'])]

            # Find the index of the series_id in the meta_info
            series_index = meta_info['series_ids'].index(str(coor_row['series_id']))

            # Get the corresponding series description
            series_desc = meta_info['series_descriptions'][series_index]

            # Update the series_desc column
            df.at[idx, 'series_desc'] = series_desc

        except KeyError:
            print(f"Error processing study_id: {coor_row['study_id']} - Study ID not found in meta_df")
            df.at[idx, 'series_desc'] = 'Unknown'
        except ValueError:
            print(f"Error processing study_id: {coor_row['study_id']} - Series ID not found in meta_df")
            df.at[idx, 'series_desc'] = 'Unknown'
        except Exception as e:
            print(f"Error processing study_id: {coor_row['study_id']} - {e}")
            df.at[idx, 'series_desc'] = 'Unknown'
    
    return df

# Apply the function
coords_with_desc = label_coords_df.copy()
coords_with_desc = add_desc(coords_with_desc, meta_df)
coords_with_desc.head(20)


,study_id,series_id,instance_number,condition,level,x,y,series_desc
0,4003253,702807833,8,Spinal Canal Stenosis,L1/L2,322.831858,227.964602,Sagittal T2/STIR
1,4003253,702807833,8,Spinal Canal Stenosis,L2/L3,320.571429,295.714286,Sagittal T2/STIR
2,4003253,702807833,8,Spinal Canal Stenosis,L3/L4,323.030303,371.818182,Sagittal T2/STIR
3,4003253,702807833,8,Spinal Canal Stenosis,L4/L5,335.292035,427.327434,Sagittal T2/STIR
4,4003253,702807833,8,Spinal Canal Stenosis,L5/S1,353.415929,483.964602,Sagittal T2/STIR
5,4003253,1054713880,4,Right Neural Foraminal Narrowing,L4/L5,187.961759,251.839388,Sagittal T1
6,4003253,1054713880,4,Right Neural Foraminal Narrowing,L5/S1,198.240918,285.613767,Sagittal T1
7,4003253,1054713880,5,Right Neural Foraminal Narrowing,L3/L4,187.227533,210.722753,Sagittal T1
8,4003253,1054713880,6,Right Neural Foraminal Narrowing,L1/L2,194.569790,127.755258,Sagittal T1
9,4003253,1054713880,6,Right Neural Foraminal Narrowing,L2/L3,191.632887,165.934990,Sagittal T1


In [6]:
sagt1_df = coords_with_desc[coords_with_desc['series_desc'] == 'Sagittal T1'].copy()
sagt2_df = coords_with_desc[coords_with_desc['series_desc'] == 'Sagittal T2/STIR'].copy()
axialt2_df = coords_with_desc[coords_with_desc['series_desc'] == 'Axial T2'].copy()

In [24]:
class AxialT2_YOLO:
    def __init__(self, df, images_dir, output_dir, img_size=384):
        """
        Initialize Axial T2 YOLO detector
        Args:
            df: DataFrame with annotations
            images_dir: Path to DICOM images
            output_dir: Path to save processed dataset
            img_size: Target image size for YOLO
        """
        self.df = df
        self.images_dir = images_dir
        self.output_dir = output_dir
        self.img_size = img_size
        
        # Create initial directory structure
        self.create_dataset_structure()
        
        # Process dataset
        self.processed_records = self.process_instance_coordinates()
        self.processed_df = self.process_spine_dataset()
        
        # Create YAML and train model
        self.yaml_path = self.create_dataset_yaml()
        self.model, self.results = self.train_yolo()

    def create_dataset_structure(self):
        """Create YOLO dataset directory structure"""
        for split in ['train', 'val']:
            for subdir in ['images', 'labels']:
                path = os.path.join(self.output_dir, split, subdir)
                os.makedirs(path, exist_ok=True)

    def process_instance_coordinates(self):
        """
        Process coordinates for Axial images, creating separate records for each level
        that has both left and right coordinates
        Returns list of records with coordinates for each complete level
        """
        result_records = []

        # Group by instance to process each image separately
        for (study_id, series_id, instance_number), instance_data in self.df.groupby(['study_id', 'series_id', 'instance_number']):
            # Process each level separately
            levels_data = {}
            
            for _, row in instance_data.iterrows():
                level = row['level']
                condition = row['condition']
                x, y = row['x'], row['y']
                
                if level not in levels_data:
                    levels_data[level] = {'left': None, 'right': None}
                
                if 'Left' in condition:
                    levels_data[level]['left'] = (x, y)
                elif 'Right' in condition:
                    levels_data[level]['right'] = (x, y)
            
            # Only create records for levels with both coordinates
            for level, coords in levels_data.items():
                if coords['left'] is not None and coords['right'] is not None:
                    record = {
                        'study_id': study_id,
                        'series_id': series_id,
                        'instance_number': instance_number,
                        'level': level,
                        'left_coord': coords['left'],
                        'right_coord': coords['right']
                    }
                    result_records.append(record)
        
        # Print statistics
        print("\nDataset Statistics:")
        total_complete = len(result_records)
        
        print(f"Total complete level pairs: {total_complete}")
        print("\nBy level statistics:")
        
        for level in ['L1/L2', 'L2/L3', 'L3/L4', 'L4/L5', 'L5/S1']:
            level_count = sum(1 for r in result_records if r['level'] == level)
            if total_complete > 0:
                percentage = (level_count / total_complete) * 100
            else:
                percentage = 0
            print(f"{level}: {level_count} complete pairs ({percentage:.1f}%)")
        
        return result_records

    def create_yolo_annotation(self, record, image_width, image_height):
        """
        Create YOLO format annotations for a specific level
        Returns list of annotations for left and right sides
        """
        annotations = []
        box_width = 0.05
        box_height = 0.05
        
        # Both coordinates must be present
        x, y = record['left_coord']
        x_norm = x / image_width
        y_norm = y / image_height
        annotations.append(f"0 {x_norm:.6f} {y_norm:.6f} {box_width:.6f} {box_height:.6f}")
        
        x, y = record['right_coord']
        x_norm = x / image_width
        y_norm = y / image_height
        annotations.append(f"1 {x_norm:.6f} {y_norm:.6f} {box_width:.6f} {box_height:.6f}")
        
        return annotations

    def process_spine_dataset(self):
        """Process and save dataset in YOLO format"""
        # Split studies
        studies = set(record['study_id'] for record in self.processed_records)
        train_studies, val_studies = train_test_split(list(studies), train_size=0.8, random_state=42)
        
        processed_counts = {'train': 0, 'val': 0}
        failed_cases = []
        
        for record in tqdm(self.processed_records, desc="Processing Axial images"):
            try:
                study_id = str(int(record['study_id']))
                series_id = str(int(record['series_id']))
                instance_number = str(int(record['instance_number']))
                level = record['level'].lower().replace('/', '_')
                
                # Construct image path
                img_path = os.path.join(self.images_dir, study_id, series_id, f"{instance_number}.dcm")
                if not os.path.exists(img_path):
                    img_path = os.path.join(self.images_dir, study_id, series_id, instance_number)
                    if os.path.exists(img_path + '.dcm'):
                        img_path = img_path + '.dcm'
                    else:
                        raise FileNotFoundError(f"Image not found: {img_path}")
                
                # Read and process image
                ds = pydicom.dcmread(img_path)
                image = ds.pixel_array
                h, w = image.shape
                
                # Create annotations
                annotations = self.create_yolo_annotation(record, w, h)
                
                # Prepare image
                image_normalized = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
                image_resized = cv2.resize(image_normalized, (self.img_size, self.img_size))
                
                # Determine split
                is_train = record['study_id'] in train_studies
                split = 'train' if is_train else 'val'
                
                # Create unique filenames including level information
                base_filename = f"{study_id}_{series_id}_{instance_number}_{level}"
                img_filename = f"{base_filename}.png"
                label_filename = f"{base_filename}.txt"
                
                # Save files
                cv2.imwrite(os.path.join(self.output_dir, split, 'images', img_filename), 
                           image_resized)
                with open(os.path.join(self.output_dir, split, 'labels', label_filename), 'w') as f:
                    f.write('\n'.join(annotations))
                
                processed_counts[split] += 1
                
            except Exception as e:
                failed_cases.append((study_id, series_id, instance_number, str(e)))
        
        print(f"\nProcessing Summary:")
        print(f"Training images: {processed_counts['train']}")
        print(f"Validation images: {processed_counts['val']}")
        
        if failed_cases:
            print("\nFailed cases:")
            for case in failed_cases:
                print(f"Study {case[0]}, Series {case[1]}, Instance {case[2]}: {case[3]}")
        
        return pd.DataFrame(self.processed_records)

    def create_dataset_yaml(self):
        """Create YOLO dataset configuration file"""
        yaml_content = {
            'path': os.path.abspath(self.output_dir),
            'train': 'train/images',
            'val': 'val/images',
            'nc': 2,  # number of classes (left and right)
            'names': {
                0: 'left',
                1: 'right'
            }
        }

        yaml_path = os.path.join(self.output_dir, 'dataset.yaml')
        with open(yaml_path, 'w') as f:
            yaml.dump(yaml_content, f, sort_keys=False)

        return yaml_path

    def train_yolo(self):
        """Train YOLO model"""
        try:
            model = YOLO('https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11x.pt')
            
            config = {
                'data': self.yaml_path,
                'imgsz': self.img_size,
                'batch': 16,
                'epochs': 120,
                'patience': 15,
                'device': '0',
                'workers': 8,
                'project': 'spine_detection',
                'name': 'axial_t2_yolo',
                'exist_ok': True,
                'pretrained': True,
                'optimizer': 'AdamW',
                'verbose': True,
                'seed': 42,
                'deterministic': True,
                'dropout': 0.2,
                'lr0': 0.001,
                'lrf': 0.01,
                'momentum': 0.937,
                'weight_decay': 0.0005,
                'warmup_epochs': 10,
                'warmup_momentum': 0.8,
                'box': 7.5,
                'cls': 0.5,
                'dfl': 1.5,
                'close_mosaic': 10,
                'amp': True
            }
            
            results = model.train(**config)
            return model, results
            
        except Exception as e:
            print(f"Error training model: {str(e)}")
            return None, None

In [25]:
# Axial T2
images_dir  = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images'
output_dir = '/kaggle/working/spine_dataset_axialt2'

axial_t2_model = AxialT2_YOLO(
    df=axialt2_df,
    images_dir=images_dir,
    output_dir=output_dir,
    img_size=384
)



Dataset Statistics:
Total complete level pairs: 5459

By level statistics:
L1/L2: 1046 complete pairs (19.2%)
L2/L3: 1116 complete pairs (20.4%)
L3/L4: 1122 complete pairs (20.6%)
L4/L5: 1094 complete pairs (20.0%)
L5/S1: 1081 complete pairs (19.8%)


Processing Axial images: 100%|██████████| 5459/5459 [00:58<00:00, 93.46it/s] 


Processing Summary:
Training images: 4332
Validation images: 1127



100%|██████████| 109M/109M [00:00<00:00, 394MB/s] 


Ultralytics 8.3.48 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=weights/yolo11x.pt, data=/kaggle/working/spine_dataset_axialt2/dataset.yaml, epochs=120, time=None, patience=15, batch=16, imgsz=384, save=True, save_period=-1, cache=False, device=0, workers=8, project=spine_detection, name=axial_t2_yolo, exist_ok=True, pretrained=True, optimizer=AdamW, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.2, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_c

...

     51/120      7.25G      1.533      1.095     0.9005         36        384: 100%|██████████| 271/271 [02:51<00:00,  1.58it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 36/36 [00:16<00:00,  2.14it/s]

                   all       1127       2254      0.717      0.746      0.798      0.387
EarlyStopping: Training stopped early as no improvement observed in last 15 epochs. Best results observed at epoch 36, best model saved as best.pt.
To update EarlyStopping(patience=15) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



51 epochs completed in 2.714 hours.
Optimizer stripped from spine_detection/axial_t2_yolo/weights/last.pt, 114.4MB
Optimizer stripped from spine_detection/axial_t2_yolo/weights/best.pt, 114.4MB

Validating spine_detection/axial_t2_yolo/weights/best.pt...
Ultralytics 8.3.48 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLO11x summary (fused): 464 layers, 56,829,334 parameters, 0 gradients, 194.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 36/36 [00:17<00:00,  2.03it/s]


                   all       1127       2254        0.8      0.771      0.842      0.398
                  left       1127       1127      0.828      0.761      0.858      0.409
                 right       1127       1127      0.772      0.781      0.825      0.387
Speed: 0.1ms preprocess, 12.2ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to spine_detection/axial_t2_yolo


In [ ]:
def save_trained_model(model, best_model_path, model_type, save_path='/kaggle/working/'):
    """
    Save the trained YOLO model with simple fixed naming
    
    Args:
        model: YOLO model object
        best_model_path: Path to the best model weights
        model_type: String indicating model type ('sag_t1', 'sag_t2', or 'axial_t2')
        save_path: Base path to save the model
    """
    # Define simple model names
    model_names = {
        'sag_t1': 'sagittal_t1_spine_detector.pt',
        'sag_t2': 'sagittal_t2_spine_detector.pt',
        'axial_t2': 'axial_t2_spine_detector.pt'
    }
    
    if model_type not in model_names:
        raise ValueError(f"Invalid model_type: {model_type}. Must be one of {list(model_names.keys())}")
    
    try:
        if not os.path.exists(best_model_path):
            print(f"Best model weights not found at {best_model_path}")
            return
            
        final_save_path = os.path.join(save_path, model_names[model_type])
        
        # Copy the model file
        import shutil
        shutil.copy(best_model_path, final_save_path)
        print(f"Model saved to {final_save_path}")
        
    except Exception as e:
        print(f"Error saving model: {str(e)}")

Save models with appropriate type
save_trained_model(
   sag_t1_model.model, 
   '/kaggle/working/spine_detection/sagittal_t1_yolo/weights/best.pt',
   model_type='sag_t1'
)

save_trained_model(
   sag_t2_model.model, 
   '/kaggle/working/spine_detection/sagittal_t2_yolo/weights/best.pt',
   model_type='sag_t2'
)

save_trained_model(
    axial_t2_model.model, 
    '/kaggle/working/spine_detection/axial_t2_yolo/weights/best.pt',
    model_type='axial_t2'
)

Model saved to /kaggle/working/axial_t2_spine_detector.pt


In [32]:
!rm -r yolo11n.pt

rm: cannot remove 'yolo11n.pt': No such file or directory


In [36]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Function to plot training and validation loss and accuracy
def plot_training_results(results):
    """
    Plot training and validation loss and metrics.

    Args:
        results: Training results object containing metrics (e.g., results.history)
    """
    if results is None:
        print("No training results available to plot.")
        return

    epochs = range(1, len(results['train_loss']) + 1)

    # Plot Loss
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, results['train_loss'], label='Train Loss')
    plt.plot(epochs, results['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()

    # Plot Metrics (Accuracy or F1-Score)
    plt.subplot(1, 2, 2)
    plt.plot(epochs, results['train_accuracy'], label='Train Accuracy')
    plt.plot(epochs, results['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()



In [37]:
def plot_axial_predictions(image_path, model_path='axial_t2_spine_detector.pt', 
                         conf_threshold=0.25, iou_threshold=0.45, img_size=384):
    """
    Plot YOLO predictions for axial images showing left and right sides
    
    Args:
        image_path: Path to DICOM image
        model_path: Path to saved YOLO model
        conf_threshold: Confidence threshold for predictions
        iou_threshold: IOU threshold for NMS
        img_size: Image size for model input
    """
    # Load model
    model = YOLO(model_path)
    
    # Read DICOM
    ds = pydicom.dcmread(image_path)
    image = ds.pixel_array
    
    # Normalize and resize
    image_normalized = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
    image_resized = cv2.resize(image_normalized, (img_size, img_size))
    
    # Convert grayscale to RGB
    image_rgb = np.stack([image_resized] * 3, axis=-1)
    
    # Create figure
    plt.figure(figsize=(15, 7))
    
    # Plot original image
    plt.subplot(1, 2, 1)
    plt.imshow(image_resized, cmap='gray')
    plt.title('Original Image')
    plt.axis('off')
    
    # Plot image with predictions
    plt.subplot(1, 2, 2)
    plt.imshow(image_resized, cmap='gray')
    plt.title('Predictions')
    
    # Get predictions
    results = model.predict(
        source=image_rgb,
        conf=conf_threshold,
        iou=iou_threshold
    )
    
    # Define colors and names for left/right sides
    side_colors = {'left': 'red', 'right': 'blue'}
    side_names = {0: 'Left', 1: 'Right'}
    
    if results[0].boxes is not None:
        boxes = results[0].boxes.cpu().numpy()
        
        # Sort boxes by x-coordinate (left to right)
        box_data = []
        for box in boxes:
            cls_id = int(box.cls[0])
            conf = box.conf[0]
            x1, y1, x2, y2 = box.xyxy[0]
            box_data.append((x1, cls_id, conf, x1, y1, x2, y2))
        
        box_data.sort()  # Sort by x1 coordinate
        
        # Plot each detection
        for i, (_, cls_id, conf, x1, y1, x2, y2) in enumerate(box_data):
            color = side_colors['left'] if cls_id == 0 else side_colors['right']
            side_name = side_names[cls_id]
            
            # Draw bounding box
            plt.gca().add_patch(plt.Rectangle(
                (x1, y1), x2-x1, y2-y1,
                fill=False, color=color, linewidth=2
            ))
            
            # Add label
            plt.text(
                x2 + 5, (y1 + y2) / 2, 
                f'{side_name}: {conf:.2f}',
                color=color, fontsize=8, verticalalignment='center',
                bbox=dict(facecolor='white', alpha=0.7, edgecolor='none')
            )
            
            # Print detection info
            print(f"Found {side_name} side with confidence {conf:.2f}")
    else:
        print("No detections found")
    
    plt.axis('off')
    plt.tight_layout()
    plt.show()



0: 224x224 1 left, 322.4ms
Speed: 2.7ms preprocess, 322.4ms inference, 10.8ms postprocess per image at shape (1, 3, 224, 224)
Error processing /kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/3481971518/12.dcm: only one element tensors can be converted to Python scalars

0: 224x224 1 left, 267.8ms
Speed: 0.7ms preprocess, 267.8ms inference, 1.4ms postprocess per image at shape (1, 3, 224, 224)
Error processing /kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/3481971518/18.dcm: only one element tensors can be converted to Python scalars

0: 224x224 1 left, 1 right, 270.3ms
Speed: 1.1ms preprocess, 270.3ms inference, 1.3ms postprocess per image at shape (1, 3, 224, 224)
Error processing /kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/3481971518/9.dcm: only one element tensors can be converted to Python scalars

0: 224x224 2 lefts, 1 right, 254.6ms
Speed: 0.9ms preprocess, 254.

In [47]:
import torch

# Clear GPU memory
torch.cuda.empty_cache()
